In [18]:
import sqlite3

In [19]:
import os

In [20]:
conn = sqlite3.connect('dedup.db')

In [21]:
c = conn.cursor()

In [24]:
c.execute('''
SELECT fingerprint,GROUP_CONCAT(ino,','),GROUP_CONCAT(lbn,','), GROUP_CONCAT(pbn,',') from ino_lbn group by fingerprint having count(*)>1  order by fingerprint 
''')
result = c.fetchall()
result = sorted(result, key=lambda item: len(item[1].split(',')),reverse=True)

In [14]:
c.execute('''
SELECT fingerprint,GROUP_CONCAT(ino,','),GROUP_CONCAT(lbn,','), GROUP_CONCAT(pbn,',') from ino_lbn group by fingerprint  order by fingerprint 
''')
result = c.fetchall()
result = sorted(result, key=lambda item: len(item[1].split(',')),reverse=True)

In [28]:
c.execute('''
SELECT * from ino_lbn
''')
result = c.fetchall()

In [26]:
c.execute('''
INSERT INTO ino_lbn (ino, lbn, fingerprint) VALUES (?, ? , ?)
''',(12345,54321,"1234556787asdas"))

In [30]:
print(len(result))
print(result)

9
[(5, 0, 5633, 'b7995faf4d84f5db4d9642b39c8255d4'), (5, 1, 5634, '0e4c6acd5fc4f25eee3648e2510b6c8c'), (5, 2, 5635, 'e638d2107782bf4a871de9d680f9b7a4'), (5, 3, 5636, '78f18668f5880762147cbbbfdbe85742'), (6, 0, 5637, '305e8101f6d4a9602c3458ce8842a5db'), (6, 1, 5638, '1b79a5bc04ad22c5ab4fee6fe040145c'), (6, 2, 5639, '1b760d6cf2b6dc949a5073118e7dc702'), (6, 3, 5640, '0425687c775ee83f395b4de083f7d9c6'), (12345, 54321, None, '1234556787asdas')]


In [ ]:
acc_block_count = 0
for i in range(0,100):
        acc_block_count = acc_block_count + len(result[i][1].split(','))
        print(i, ": fingerprint:", result[i][0]," count:",len(result[i][1].split(',')),"acc_count:",acc_block_count)
        
        ino_list = result[i][1].split(',')
        lbn_list = result[i][2].split(',')
        for j in range(0,min(5,len(ino_list))):
            print(f'[{ino_list[j]},{lbn_list[j]}]', end=' ')
        print()

In [158]:
acc_block_count = 0
same_pos_cnt = 0
near_file_cnt = 0
locality = 0
for i in range(0,len(result)):
        acc_block_count = acc_block_count + len(result[i][1].split(','))
        #print(i, ": fingerprint:", result[i][0]," count:",len(result[i][1].split(',')),"acc_count:",acc_block_count)
        ino_list = result[i][1].split(',')
        lbn_list = result[i][2].split(',')
        same_pos = True
        near_file = True 
        for lbn in lbn_list:
            if(lbn!=lbn_list[0]):
                same_pos = False
                break
        min_ino = 9999999
        max_ino = 0
        for ino in ino_list:
            min_ino = min(int(min_ino),int(ino))
            max_ino = max(int(max_ino),int(ino))
        if(max_ino-min_ino>10000):
            #print(result[i]," ",max_ino," ",min_ino)
            near_file = False
        if(same_pos):
            same_pos_cnt += 1 
        if(near_file):
            near_file_cnt += 1
        if(same_pos and near_file):
            locality += 1 
print(len(result))
print(same_pos_cnt)
print(near_file_cnt)
print(locality)

1041195
995935
702350
687377


In [ ]:
same_file_ino_set = set()


acc_block_count = 0
same_file_pair = 0
similiar_file_pair = 0
all_file_pair = 0
encountered_file_ino = []
files_size = 0
similiar_files_size = 0
for i in range(0,len(result)):
        ino_list = result[i][1].split(',')
        lbn_list = result[i][2].split(',')
        count = len(lbn_list)
        if(count==2):
            ino_1 = ino_list[0]
            ino_2 = ino_list[1] 
            if(ino_1 in encountered_file_ino or ino_2 in encountered_file_ino):
                continue
            all_file_pair = all_file_pair + 1
            file_1 = os.popen(f"find /mnt/android/f2fs/ -inum {ino_1}").read().rstrip('\n')
            file_2 = os.popen(f"find /mnt/android/f2fs/ -inum {ino_2}").read().rstrip('\n')
            #files_size = files_size + os.stat(file_2).st_size
            print("f1:", file_1)
            print("f2:", file_2)
            md5_1 = os.popen(f"md5sum {file_1}").read().split(" ")[0]
            md5_2 = os.popen(f"md5sum {file_2}").read().split(" ")[0]
            if(md5_1 == md5_2):
                same_file_pair = same_file_pair + 1
                files_size = files_size + os.stat(file_1).st_size*2
                print("same")
            else:
                similiar_file_pair = similiar_file_pair + 1 
                print("similiar")
                similiar_files_size = similiar_files_size + os.stat(file_1).st_size + os.stat(file_2).st_size
            encountered_file_ino.append(ino_1)
            encountered_file_ino.append(ino_2)
print("all_file_pair:",all_file_pair)
print("same_file_pair:",same_file_pair)
print("similiar_file_pair:",similiar_file_pair)
print("files_size:" , files_size/1024/1024)
print("similiar_size:", similiar_files_size/1024/1024)

In [15]:
c.execute('''
SELECT fingerprint,GROUP_CONCAT(ino,','),GROUP_CONCAT(du,',') from ino  group by fingerprint  order by fingerprint 
''')
result = c.fetchall()
#result = sorted(result, key=lambda item: len(item[1].split(',')),reverse=True)

In [149]:
c.execute('''
SELECT * from ino  
''')
result = c.fetchall()

In [152]:
print(len(result))
print(result[1])

124142
('0000a4dd477a5ec99572c040f7e1c2e9', '72104', '32768')


In [16]:
for i in range(0,len(result)):
    miss_flag = False
    du_arr = result[i][2].split(',')
    disk_space = 0
    for j in range(1,len(du_arr)):
        disk_space += int(du_arr[j])
    #assert(du_arr[0]==du_arr[1])
    disk_space_t = (disk_space//1024//1024,)
    result[i] = result[i] + disk_space_t
result = sorted(result, key=lambda item:item[3], reverse=True)

In [18]:
top_dup_file_count = 0
dup_file_size = 0
for i in range(0,5000):
    if(len(result[i][1].split(','))>1):
        top_dup_file_count += 1
        dup_file_size += result[i][3]
        print(i,":",result[i])
print(top_dup_file_count)
print(dup_file_size)

0 : ('b5cfa9d6c8febd618f91ac2843d50a1c', '7154,8518,9067,13663,18093,18654,18851,29250,30785,35815,36881,66336,69545,96597,102789,103446,104384,105147,106030,106032,106048,108485,109576,110832,115476,115625,120476,121634,126045,126865,127249,128126,130417,134015,134490', '4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400,4198400', 136)
1 : ('5ef2b6dfae03f7eea006f17ea9398e7d', '64869,127131', '100536320,100536320', 95)
2 : ('7bbdabd366e38eda2626b32f102ad2da', '12318,12537', '98729984,98729984', 94)
3 : ('fdc48fea9af9321b1856bc44eb0c2afd', '92804,109481', '53018624,53018624', 50)
4 : ('8b522444d1bead4297ba297bcc64b7ff', '22688,92109', '48279552,48279552', 46)
5 : ('2bdf82349d757d8a431e9d333771f3f9', '25879,129680,130497', '20451328,20451328,20451328', 39)
6 : ('8a3016a8

In [19]:
unique_file_size = 0
file_size_acc = 0
print(len(result))
for i in range(0,len(result)):
    if(len(result[i][1].split(','))>1):
        #file_path = os.popen(f'find /mnt/android/f2fs/ -inum {arr[0]}').read().rstrip('\n')
        #disk_space = os.popen(f'du -s --block-size=1 "{file_path}"').read().rstrip('\n').split()[0]
        miss_flag = False
        du_arr = result[i][2].split(',')
        for j in du_arr:
            if(j!=du_arr[0]):
                #print(result[i])
                miss_flag = True
                break
        if(miss_flag ==True):
            continue
        #assert(du_arr[0]==du_arr[1])
        disk_space = int(du_arr[0])
        unique_file_size = unique_file_size + disk_space
        file_size_acc = file_size_acc + disk_space*len(du_arr)
        #stat_info = os.stat(file_path)
        #unique_file_size += stat_info.st_blocks * stat_info.st_blksize
        #file_size_acc = file_size_acc + stat_info.st_blocks * stat_info.st_blksize*len(arr)

print(file_size_acc)
print(unique_file_size)

73862
2834344960
1239291392


In [97]:
print((8580424704-4093075456)//1024//1024)

4279


In [171]:
print(33815-28431-740-4279)

365
